# Writing a Webserver in Flask

In this document we will look at writing a webserver in Python using Flask. You will need the files in the templates directory of the SWS3009Lect4.zip file.

## Installing Flask

We start by installing FLask:

In [1]:
! pip3 install flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [flask]


## Importing the various libraries

We now import the following libraries from the flask workspace:

- Flask: The main Flask object
- request: To handle requests from the client
- render_template: Lets us render web pages using Jinja

We also import MongoClient from pymongo to let us connect to the MongoDB server.


In [3]:
from flask import Flask, request, render_template
from pymongo import MongoClient

# Create the Flask object
app = Flask(__name__)


## Creating the Routes

We now come to the main part of our web server - the various routes. A "route" essentially defines an end-point for our connection. For example if we want to connect to http://mysite.com/hello, then we need to create a /hello route. Likewise if we just wanted to connect to http://mysite.com/, we need to create a "/" route.

### The "root" route

We start witht his simple "/" route where we just print out a message. Since we are not sending any data to this route, we define it as a simple GET route.

Note that we must also return a code. Here 200 means "Success".

In [4]:
@app.route('/', methods = ['GET'])
def root():
    return 'SWS3009 Sample Site', 200

### The "/index" route

In the /index route (called using http://mysite.com/index, for example), we will see how to use the request object to extract data that has been sent as part of a GET request. E.g. using http://mysite.com/index?name=polar%20%bear.

We extract parameters sent using GET by making use of request.args.get, as shown here.

We also see an example of how to use render_template to render the index.html Jinja script in the templates directory.

In [5]:
# Examples of how to render a template. Also note
# how we use requests.args.get to extract GET parameters
@app.route('/index', methods = ['GET'])
def index():
    """ Demo routine to show how to pass parameters through GET """

    # Extract GET parameters from request object
    name = request.args.get('name')

    if name is None:
        name = 'Bob Jones'

    return render_template('index.html', info = {"title":"Hello World", "name":name}), 200

### The /put route

We now create a new /put endpoint, this time using a POST method. We see an example of how to extract data sent via a POST request by using request.get_json(). 

In [8]:
# Example of how to handle JSON sent in via POST
@app.route('/put', methods = ['POST'])
def put_record():
    """ Add a new record to the database """

    try:
        new_rec = request.get_json()

        print(new_rec)
        if new_rec is not None:
            col.insert_one(new_rec)

        return 'OK', 200 
    except Exception as e:
        return e, 400 

### The /get endpoint

Just a simple endpoint to list every item in the database.

In [9]:
@app.route('/get', methods = ['GET'])
def get():
    """ Get all records and return it """

    results = col.find()

    return render_template('get.html', results = results)

## Main

Finally our main simply connects to the MongoDB backend, then calls app.run to start our database server on port 3237. The 'host="0.0.0.0"' option allows any client to connect to our web server. Otherwise connections are restricted only to clients running locally.

In [ ]:
# Main code

def main():
    global client, db, col, app
    client = MongoClient('mongodb://localhost:27017/')
    db = client['my_db']
    col = db['MyCollection']
    app.run(port = 3237, host="0.0.0.0")

if __name__ == '__main__':
    main()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3237
 * Running on http://172.31.71.96:3237
Press CTRL+C to quit


{'name': 'Document 1', 'Description': 'First test document'}


[2025-07-04 23:18:21,576] ERROR in app: Exception on /put [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 939, in finalize_request
    response = self.make_response(rv)
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 1249, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The return type must be a string, dict, list, tuple with headers or status, Response instance, or WSGI callable, but it was a ServerSelectionTimeoutError.
127.0.0.1 - - [04/Jul/2025 23:18:21] "POST /put HTTP/1.1" 500 -


{'id': 1, 'Why': 'We can do different formats!'}


[2025-07-04 23:18:52,893] ERROR in app: Exception on /put [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 939, in finalize_request
    response = self.make_response(rv)
  File "/opt/anaconda3/envs/tfenv/lib/python3.9/site-packages/flask/app.py", line 1249, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The return type must be a string, dict, list, tuple with headers or status, Response instance, or WSGI callable, but it was a ServerSelectionTimeoutError.
127.0.0.1 - - [04/Jul/2025 23:18:52] "POST /put HTTP/1.1" 500 -
